# AER - Access Entitlement Review Reporter (Optimized v3)

**Updates:**
- **Tree Selector**: Pick specific Apps under Categories (Q3, Q4).
- **Integrated Reporting**: Reports generated immediately after processing in `output/YYYY-MM-DD/`.
- **Detailed Logging**: Shows user count per App & success logs per user.
- **UI Improvements**: Better readability for Email rows.

In [1]:
# === Cell 1: 設定與認證 (互動式視窗登入 & UTF-8 Logging) ===
import os
import sys
import io
import logging
from datetime import datetime
from dotenv import load_dotenv
from msal import PublicClientApplication
import requests

# 讀取 .env
load_dotenv()

# === Logging 設定 ===
os.makedirs("output", exist_ok=True)
log_file = f"output/aer_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

logger = logging.getLogger("aer")
logger.handlers.clear()
logger.setLevel(logging.INFO)

def _get_console_stream():
    s = getattr(sys, "stdout", None)
    try:
        if s and hasattr(s, "reconfigure"):
            s.reconfigure(encoding="utf-8")
            if hasattr(sys.stderr, "reconfigure"):
                sys.stderr.reconfigure(encoding="utf-8")
            return s
    except Exception: pass
    try:
        buf = getattr(s, "buffer", None)
        if buf: return io.TextIOWrapper(buf, encoding="utf-8", errors="replace")
    except Exception: pass
    return None

formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')
ch = logging.StreamHandler(_get_console_stream())
ch.setFormatter(formatter)
logger.addHandler(ch)
fh = logging.FileHandler(log_file, encoding="utf-8")
fh.setFormatter(formatter)
logger.addHandler(fh)

# === Azure AD ===
TENANT_ID = os.getenv("AZURE_TENANT_ID")
CLIENT_ID = os.getenv("AZURE_CLIENT_ID")
SHAREPOINT_HOST = os.getenv("SHAREPOINT_HOST", "davidshih.sharepoint.com")
SITE_NAME = os.getenv("SITE_NAME", "aer")
APP_NAME = "2025 Entitlement Review"
BASE_PATH = APP_NAME
SENDER_EMAIL = os.getenv("SENDER_EMAIL")

SCOPES = [
    "User.Read.All",         # 讀取你的個人檔案
    "Mail.Send",
    "Mail.Read",         # 讀取你的 Email (Copilot 核心功能)
    "Files.Read.All",    # 讀取 OneDrive/SharePoint 檔案
    "Sites.Read.All",    # 搜尋 SharePoint 網站
    "People.Read.All", # 備用
    "OnlineMeetingTranscript.Read.All",
    "Chat.Read",
    "ChannelMessage.Read.All",
    "ExternalItem.Read.All"
]

app = PublicClientApplication(CLIENT_ID, authority=f"https://login.microsoftonline.com/{TENANT_ID}")

print("🚀 正在開啟瀏覽器進行登入...")
interactive_result = app.acquire_token_interactive(scopes=SCOPES, prompt="select_account")

if "access_token" not in interactive_result:
    raise RuntimeError(f"登入失敗: {interactive_result.get('error_description')}")

headers = {"Authorization": f"Bearer {interactive_result['access_token']}"}
logger.info("✅ 登入成功")
logger.info(f"Log 檔案: {log_file}")

🚀 正在開啟瀏覽器進行登入...


2026-01-09 13:10:31,152 | INFO | ✅ 登入成功
2026-01-09 13:10:31,158 | INFO | Log 檔案: output/aer_20260109_131027.log


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import requests
import json
from datetime import datetime

# ==========================================
# 1. 定義 Graph API 功能 (M365 Data Skills)
# ==========================================

def get_latest_emails():
    """ 模仿 Copilot: 'Summarize my emails' 的前置步驟 - 抓取資料 """
    url = "https://graph.microsoft.com/v1.0/me/messages?$top=3&$select=subject,from,receivedDateTime,bodyPreview"
    try:
        response = requests.get(url, headers=headers) # 使用 Cell 1 的 headers
        if response.status_code == 200:
            emails = response.json().get('value', [])
            if not emails: return "📭 最近沒有新郵件。"
            
            html = "<ul style='padding-left:15px; margin:5px 0;'>"
            for mail in emails:
                subject = mail.get('subject', '無主旨')
                sender = mail.get('from', {}).get('emailAddress', {}).get('name', '未知')
                preview = mail.get('bodyPreview', '')[:50] + "..."
                html += f"<li><b>[{sender}]</b> {subject}<br><span style='color:grey;font-size:12px'>{preview}</span></li>"
            html += "</ul>"
            return f"📧 <b>這是您最近的 3 封郵件：</b>{html}"
        else:
            return f"❌ 讀取郵件失敗: {response.status_code}"
    except Exception as e:
        return f"💥 Error: {str(e)}"

def get_recent_files():
    """ 模仿 Copilot: 'What files did I work on?' """
    # 抓取 "Recent" 檔案列表
    url = "https://graph.microsoft.com/v1.0/me/drive/recent?$top=5"
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            files = response.json().get('value', [])
            if not files: return "📂 最近沒有開啟過的檔案。"
            
            html = "<ul style='padding-left:15px; margin:5px 0;'>"
            for f in files:
                name = f.get('name')
                web_url = f.get('webUrl')
                html += f"<li><a href='{web_url}' target='_blank'>{name}</a></li>"
            html += "</ul>"
            return f"📂 <b>您最近使用的檔案：</b>{html}"
        else:
            return f"❌ 讀取檔案失敗: {response.status_code}"
    except Exception as e:
        return f"💥 Error: {str(e)}"

def get_my_profile():
    url = "https://graph.microsoft.com/v1.0/me"
    try:
        r = requests.get(url, headers=headers)
        if r.status_code == 200:
            data = r.json()
            return f"👤 <b>{data.get('displayName')}</b> ({data.get('jobTitle')})<br>{data.get('mail')}"
        return "❌ 無法讀取個人檔案"
    except: return "💥 Error"

# ==========================================
# 2. 模擬 Copilot 的意圖判斷 (Router)
# ==========================================
def m365_copilot_router(user_text):
    """
    這裡扮演 Copilot 的 'Orchestrator' 角色。
    分析使用者意圖 -> 呼叫對應 Graph API -> 回傳結果
    """
    txt = user_text.lower()
    
    # 規則 1: 查信
    if "email" in txt or "信" in txt or "郵件" in txt:
        return get_latest_emails()
    
    # 規則 2: 查檔案
    elif "file" in txt or "檔案" in txt or "document" in txt:
        return get_recent_files()
    
    # 規則 3: 查自己
    elif "who am i" in txt or "我是誰" in txt:
        return get_my_profile()
        
    # 預設回應
    else:
        return (f"🤖 我是您的 Python M365 助手。<br>"
                f"因為使用 <b>Delegated Permission</b>，我可以存取您的真實資料。<br>"
                f"試試看輸入：<br>"
                f"👉 <b>我的最新郵件</b><br>"
                f"👉 <b>最近使用的檔案</b><br>"
                f"👉 <b>我是誰</b>")

# ==========================================
# 3. UI 顯示 (與之前相同，稍微美化)
# ==========================================
chat_history_style = """
<style>
.chat-box { font-family: 'Segoe UI', sans-serif; padding: 10px; height: 400px; overflow-y: auto; border: 1px solid #e1dfdd; border-radius: 4px; background: white;}
.user-row { text-align: right; margin: 10px 0; }
.user-msg { background-color: #0078D4; color: white; padding: 8px 12px; border-radius: 15px 15px 0 15px; display: inline-block; text-align: left; box-shadow: 1px 1px 2px rgba(0,0,0,0.2); }
.bot-row { text-align: left; margin: 10px 0; }
.bot-msg { background-color: #f3f2f1; color: #201f1e; padding: 10px 12px; border-radius: 15px 15px 15px 0; display: inline-block; border: 1px solid #e1dfdd; max-width: 80%; }
a { color: #0078D4; text-decoration: none; font-weight: 500; } a:hover { text-decoration: underline; }
</style>
<div class='chat-box'>
"""

output_html = widgets.HTML(value=chat_history_style + "</div>")
input_text = widgets.Text(placeholder="輸入 '我的郵件' 或 '我的檔案'...", layout=widgets.Layout(width='70%'))
btn_send = widgets.Button(description="發送", button_style='primary', icon='location-arrow', layout=widgets.Layout(width='20%'))
loader = widgets.HTML(value="", layout=widgets.Layout(height='25px'))

# 狀態變數
chat_content = chat_history_style

def on_submit(b):
    global chat_content
    user_say = input_text.value.strip()
    if not user_say: return
    
    # User UI
    chat_content += f"<div class='user-row'><span class='user-msg'>{user_say}</span></div>"
    output_html.value = chat_content + "</div>"
    input_text.value = ""
    loader.value = "<span style='color:#0078D4; font-weight:bold;'>🔵 Copilot 正在存取 Graph API...</span>"
    btn_send.disabled = True
    
    # Backend Logic
    reply = m365_copilot_router(user_say)
    
    # Bot UI
    chat_content += f"<div class='bot-row'><span class='bot-msg'>{reply}</span></div>"
    output_html.value = chat_content + "</div>"
    loader.value = ""
    btn_send.disabled = False

btn_send.on_click(on_submit)
input_text.on_submit(on_submit)

display(widgets.VBox([
    widgets.HTML("<h3>🚀 Jupyter M365 Copilot (Delegated Access)</h3>"),
    output_html,
    loader,
    widgets.HBox([input_text, btn_send], layout=widgets.Layout(justify_content='space-between'))
]))

In [ ]:
# === Cell 2: SharePoint API & App Tree Selector (Combined) ===
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 1. SharePoint API Functions ---
def get_site_id(site_name: str) -> str:
    url = f"https://graph.microsoft.com/v1.0/sites/{SHAREPOINT_HOST}:/sites/{site_name}"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()["id"]

def list_folders(site_id: str, path: str) -> list[dict]:
    if not path or path.strip() == "":
        url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root/children"
    else:
        clean_path = path.strip("/")
        url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{clean_path}:/children"
    resp = requests.get(url, headers=headers)
    return [
        {"name": item["name"], "webUrl": item.get("webUrl", "")}
        for item in resp.json().get("value", []) if item.get("folder")
    ]

def list_excel_files(site_id: str, folder_path: str) -> list[dict]:
    clean_path = folder_path.strip("/")
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{clean_path}:/children"
    resp = requests.get(url, headers=headers)
    files = []
    for item in resp.json().get("value", []):
        if item["name"].endswith(".xlsx"):
            files.append({
                "id": item["id"], "name": item["name"],
                "lastModifiedDateTime": item.get("lastModifiedDateTime"), "webUrl": item.get("webUrl")
            })
    return sorted(files, key=lambda f: f.get("lastModifiedDateTime", ""), reverse=True)

def download_file(site_id: str, file_path: str) -> bytes:
    clean_path = file_path.strip("/")
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/root:/{clean_path}:/content"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.content

def get_file_audit_log(site_id: str, file_id: str) -> str:
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/items/{file_id}/versions"
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200: return "無法取得版本紀錄"
    logs = []
    for v in resp.json().get("value", []):
        mod_time = v.get("lastModifiedDateTime", "")[:19].replace("T", " ")
        actor = v.get("lastModifiedBy", {}).get("user", {}).get("displayName") or "System"
        logs.append(f"{mod_time} - {actor}")
    return "\n".join(logs)

# --- 2. Initialize Connection ---
try:
    site_id = get_site_id(SITE_NAME)
    logger.info(f"✅ SharePoint 連線成功 (Site ID: {site_id[:10]}...)")
except Exception as e:
    logger.error(f"❌ 連線失敗: {e}")
    raise

# --- 3. Tree Selector UI ---
TARGET_APPS = [] # Stores (Category, AppName, Path)

def create_app_selector():
    print(f"📂 正在讀取根目錄: {BASE_PATH} ...")
    try:
        categories = list_folders(site_id, BASE_PATH)
    except Exception as e:
        print(f"❌ 讀取失敗: {e}")
        return

    ui_container = widgets.VBox()
    app_checkboxes = []
    
    # Header
    ui_container.children += (widgets.HTML("<h3>📂 App 選擇器 (請展開 Category 並勾選 App)</h3>"),)

    for cat in categories:
        # 跳過系統資料夾
        if cat['name'] in ["Forms", "_private"] or "audit" in cat['name'].lower():
            continue
            
        # Category Row
        cat_label = widgets.HTML(f"<b>📁 {cat['name']}</b>", layout=widgets.Layout(width='100px'))
        btn_expand = widgets.Button(description="➕ 展開", button_style='info', layout=widgets.Layout(width='80px'))
        app_list_box = widgets.VBox(layout=widgets.Layout(margin='0 0 0 30px', display='none')) # 預設隱藏
        
        # Logic to load apps
        def on_expand(b, cat_name=cat['name'], container=app_list_box, btn=btn_expand):
            if btn.description == "➕ 展開":
                btn.description = "⏳"
                sub_path = f"{BASE_PATH}/{cat_name}"
                try:
                    apps = list_folders(site_id, sub_path)
                    checks = []
                    if not apps:
                        checks.append(widgets.Label("(無資料夾)"))
                    
                    for app in apps:
                        if app['name'] in ["Forms", "_private"]: continue
                        cb = widgets.Checkbox(value=False, description=app['name'], indent=False)
                        cb.app_data = (cat_name, app['name'], f"{sub_path}/{app['name']}")
                        checks.append(cb)
                        app_checkboxes.append(cb)
                    
                    container.children = tuple(checks)
                    container.layout.display = 'block'
                    btn.description = "➖ 收起"
                except Exception as e:
                    container.children = (widgets.Label(f"Error: {e}"),)
                    btn.description = "❌"
            else:
                # Toggle visibility
                if container.layout.display == 'none':
                    container.layout.display = 'block'
                    btn.description = "➖ 收起"
                else:
                    container.layout.display = 'none'
                    btn.description = "➕ 展開"

        btn_expand.on_click(on_expand)
        ui_container.children += (widgets.HBox([btn_expand, cat_label]), app_list_box)

    # Footer Actions
    btn_confirm = widgets.Button(description="✅ 確認選擇", button_style='success', layout=widgets.Layout(margin='20px 0'))
    output_area = widgets.Output()

    def on_confirm(b):
        global TARGET_APPS
        TARGET_APPS = [cb.app_data for cb in app_checkboxes if cb.value]
        with output_area:
            clear_output()
            if not TARGET_APPS:
                print("⚠️ 未選擇任何 App！")
            else:
                print(f"🎯 已選擇 {len(TARGET_APPS)} 個 Apps。請執行 Cell 4 開始處理。")
                for c, a, p in TARGET_APPS[:5]:
                    print(f" - [{c}] {a}")
                if len(TARGET_APPS) > 5: print(" ...")

    btn_confirm.on_click(on_confirm)
    
    display(ui_container, btn_confirm, output_area)

create_app_selector()

In [ ]:
# === Cell 3: Excel 讀取函數 (保持不變) ===
from openpyxl import load_workbook
from io import BytesIO

COL_REVIEWER = "Reviewer"
COL_RESPONSE = "Reviewer's Response"
COL_DETAILS = "Details of Access change"

def read_visible_rows(excel_bytes: bytes, reviewer_name: str, file_name: str, folder_url: str) -> list[dict]:
    wb = load_workbook(BytesIO(excel_bytes))
    ws = wb.active
    
    header_row = [cell.value for cell in ws[1]]
    col_map = {str(name).strip(): idx for idx, name in enumerate(header_row) if name}
    
    reviewer_col = col_map.get(COL_REVIEWER)
    response_col = col_map.get(COL_RESPONSE)
    details_col = col_map.get(COL_DETAILS)
    
    if reviewer_col is None or response_col is None:
        raise ValueError(f"Missing columns. Found: {list(col_map.keys())}")
    
    has_filter = any(ws.row_dimensions[i].hidden for i in range(2, ws.max_row + 1) if i in ws.row_dimensions)
    results = []
    
    for row_idx in range(2, ws.max_row + 1):
        if has_filter and ws.row_dimensions.get(row_idx) and ws.row_dimensions[row_idx].hidden:
            continue
        row = [cell.value for cell in ws[row_idx]]
        
        reviewer = row[reviewer_col] if reviewer_col < len(row) else None
        response = row[response_col] if response_col < len(row) else None
        details = row[details_col] if details_col is not None and details_col < len(row) else None
        
        if not has_filter and (reviewer is None or str(reviewer).strip().lower() != reviewer_name.lower()):
            continue
        
        results.append({
            "reviewer": reviewer_name, "response": response, "details": details,
            "is_missing": (response is None or str(response).strip() == ""),
            "row_number": row_idx, "file_name": file_name, "folder_url": folder_url
        })
    return results

In [ ]:
# === Cell 4: 資料處理 & 報告生成 (Fetch Creation Date + Retry) ===
import pandas as pd
import re
import time
import requests
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
from IPython.display import display, HTML

# 1. Check Selection
if 'TARGET_APPS' not in globals() or not TARGET_APPS:
    logger.warning("⚠️ 請先在 Cell 2 選擇 App！")
    TARGET_APPS = []

all_responses = []
errors = []

# 2. Processing Loop
for category, current_app_name, current_path in TARGET_APPS:
    try:
        reviewers = list_folders(site_id, current_path)
        # Filter valid user folders
        reviewers = [r for r in reviewers if r['name'] not in ["Forms", "_private"] and "audit" not in r['name'].lower()]
        
        logger.info(f"🚀 Processing: {current_app_name} | Users: {len(reviewers)}")
        
        for folder in reviewers:
            reviewer_name = folder["name"]
            folder_url = folder["webUrl"]
            folder_path = f"{current_path}/{reviewer_name}"
            
            # Retry Logic: Try 2 times
            for attempt in range(2):
                try:
                    excel_files = list_excel_files(site_id, folder_path)
                    target_files = [f for f in excel_files if reviewer_name.lower() in f["name"].lower()]
                    
                    if not target_files:
                        valid_excels = [f for f in excel_files if not f["name"].startswith("~$")]
                        if not valid_excels: break # No file to read
                        target_file = valid_excels[0]
                    else:
                        target_file = target_files[0]
                    
                    file_name = target_file["name"]
                    
                    # --- NEW: Fetch specific metadata for Creation Date (Version 1) ---
                    meta_url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive/items/{target_file['id']}"
                    meta_resp = requests.get(meta_url, headers=headers)
                    created_date = meta_resp.json().get("createdDateTime") if meta_resp.status_code == 200 else target_file.get("lastModifiedDateTime")

                    # Get Audit Log
                    audit_log = get_file_audit_log(site_id, target_file["id"])
                    
                    # Download Content
                    content = download_file(site_id, f"{folder_path}/{file_name}")
                    
                    rows = read_visible_rows(content, reviewer_name, file_name, folder_url)
                    
                    for row in rows:
                        row.update({
                            "Category": category, 
                            "App_Name": current_app_name,
                            "Audit_History": audit_log, 
                            "Last_Modified": target_file.get("lastModifiedDateTime"),
                            "Created_Date": created_date # Store for Due Date calc
                        })
                    
                    all_responses.extend(rows)
                    logger.info(f"  ✅ Read: {reviewer_name} ({len(rows)})")
                    break # Exit retry loop on success

                except Exception as e:
                    if attempt == 0:
                        logger.warning(f"  ⚠️ Retry needed for {reviewer_name} (Wait 1s)...")
                        time.sleep(1)
                    else:
                        logger.error(f"  ❌ Failed {reviewer_name}: {e}")
                        errors.append({
                            "Category": category, "App_Name": current_app_name, 
                            "reviewer": reviewer_name, "error": str(e), "folder_url": folder_url
                        })
    except Exception as e:
        logger.error(f"❌ Critical App Error ({current_app_name}): {e}")

# 3. Dataframe Construction
df = pd.DataFrame(all_responses)
logger.info(f"\n=== Processing Complete === Total Records: {len(df)}")

# 4. Reporting
today_str = datetime.now().strftime("%Y-%m-%d")
output_dir = f"output/{today_str}"
os.makedirs(output_dir, exist_ok=True)
timestamp = datetime.now().strftime("%H%M%S")

def clean_filename(s): return re.sub(r'[\\/*?:"<>|]', "_", str(s))

def format_excel(path):
    try:
        wb = load_workbook(path)
        ws = wb.active
        long_cols = ["Audit_History", "Details of Access Changes", "Reviewer's Response", "error", "folder_url"]
        for col in ws.columns:
            head = str(col[0].value).strip() if col[0].value else ""
            letter = get_column_letter(col[0].column)
            if head in long_cols:
                ws.column_dimensions[letter].width = 50
                for c in col: c.alignment = Alignment(wrap_text=True, vertical='top')
            else:
                ws.column_dimensions[letter].width = 25
        wb.save(path)
    except: pass

col_mapping = {
    "Category": "Category",
    "App_Name": "App Name",
    "reviewer": "Reviewer",
    "response": "Reviewer's Response",
    "details": "Details of Access Changes",
    "is_missing": "Is Missing",
    "Audit_History": "Audit_History",
    "Last_Modified": "Last Modified",
    "Created_Date": "Created Date",
    "file_name": "File Name",
    "folder_url": "Folder Link"
}

report_html = f"<h3>📂 報告已生成 (路徑: {output_dir})</h3><ul>"

if len(df) > 0:
    export_df = df.rename(columns=col_mapping)
    export_cols = [c for c in list(col_mapping.values()) if c in export_df.columns]

    # Consolidated File
    big_file = f"{output_dir}/ALL_Consolidated_{timestamp}.xlsx"
    export_df[export_cols].to_excel(big_file, index=False)
    format_excel(big_file)
    report_html += f"<li><b>🌎 Big File:</b> <code>{big_file}</code></li>"
    
    # Individual App Files
    for app in df["App_Name"].unique():
        sub = export_df[export_df["App Name"] == app]
        cat = df[df["App_Name"] == app].iloc[0]["Category"]
        fname = f"{output_dir}/{clean_filename(cat)}_{clean_filename(app)}_{timestamp}.xlsx"
        sub[export_cols].to_excel(fname, index=False)
        format_excel(fname)
        report_html += f"<li>{app}: <code>{fname}</code></li>"
else:
    report_html += "<li style='color:red'>No data found.</li>"

if errors:
    err_file = f"{output_dir}/ERRORS_{timestamp}.xlsx"
    pd.DataFrame(errors).to_excel(err_file, index=False)
    report_html += f"<li style='color:red'><b>Errors:</b> <code>{err_file}</code></li>"

display(HTML(report_html + "</ul>"))

In [ ]:
# === Cell 5: 狀態統計表 (Clean Summary Report) ===
from IPython.display import display, HTML
import pandas as pd

if 'df' not in globals() or len(df) == 0:
    display(HTML("<p>No Data</p>"))
else:
    df['response_clean'] = df['response'].fillna("").astype(str).str.lower().str.strip()
    
    # 1. Base Aggregation
    base = df.groupby(["Category", "App_Name", "reviewer", "folder_url"]).agg(
        total=("reviewer", "count"), 
        missing=("is_missing", "sum"),
        file_date=("Last_Modified", "min") 
    ).reset_index()
    
    # 2. Response Status Counts
    status_counts = pd.pivot_table(df, index=["Category", "App_Name", "reviewer"], columns="response_clean", aggfunc="size", fill_value=0).reset_index()
    summary = pd.merge(base, status_counts, on=["Category", "App_Name", "reviewer"], how="left")
    
    # 3. Calculate Logic
    def cnt(r, keys): return sum(r.get(k, 0) for k in status_counts.columns if any(x in str(k) for x in keys))
    summary['Approved'] = summary.apply(lambda r: cnt(r, ['approv', 'retain', 'keep']), axis=1)
    summary['Denied'] = summary.apply(lambda r: cnt(r, ['denied', 'remove', 'delete']), axis=1)
    summary['Change Required'] = summary.apply(lambda r: cnt(r, ['change', 'modif']), axis=1)
    
    # 4. Status Column Logic
    def get_status(r):
        if r['missing'] > 0: return "Pending"
        if r['missing'] == 0: return "Completed"
        return "In Progress"
    summary['Status'] = summary.apply(get_status, axis=1)

    # 5. Sorting
    summary['is_red'] = summary['missing'] > 0
    summary = summary.sort_values(by=['is_red', 'App_Name', 'missing'], ascending=[False, True, False])
    
    # 6. Global var for Email Cell
    summary_df = summary.copy() 
    
    # 7. Prepare Clean Excel Output (Drop specific columns, rename headers)
    # Requested: Drop duplicated, keep Total, Missing, Approved, Denied, Change, Status
    final_output_cols = [
        "Category", "App_Name", "reviewer", 
        "total", "missing", "Approved", "Denied", "Change Required", "Status"
    ]
    
    # Rename for professional look in Excel
    rename_map = {
        "App_Name": "App Name",
        "reviewer": "Reviewer",
        "total": "Total",
        "missing": "Missing"
    }
    
    excel_export = summary[final_output_cols].rename(columns=rename_map)
    
    s_file = f"{output_dir}/Status_Summary_{timestamp}.xlsx"
    excel_export.to_excel(s_file, index=False)
    
    # 8. Render HTML Table (Keep internal logic for colors)
    html = "<table style='width:100%; border-collapse:collapse; font-family:sans-serif; font-size:12px;'>"
    html += "<tr style='background:#333; color:white;'><th>Category</th><th>App</th><th>Reviewer</th><th>Miss/Total</th><th>Breakdown</th><th>Status</th></tr>"
    
    for _, r in summary.iterrows():
        # Visual Logic
        row_bg = "#ffe6e6" if r['missing'] > 0 else "#e6ffe6"
        status_icon = "❌ Pending" if r['missing'] > 0 else "✅ Completed"
        
        html += f"<tr style='background:{row_bg}; border-bottom:1px solid #ccc;'>"
        html += f"<td style='padding:5px'>{r['Category']}</td>"
        html += f"<td style='padding:5px'><b>{r['App_Name']}</b></td>"
        html += f"<td style='padding:5px'><a href='{r['folder_url']}' target='_blank' style='text-decoration:none; color:#0066cc'>{r['reviewer']}</a></td>"
        html += f"<td style='padding:5px; text-align:center'><b style='color:red'>{r['missing']}</b> / {r['total']}</td>"
        html += f"<td style='padding:5px'>Approve:{r['Approved']} | Deny:{r['Denied']} | Change:{r['Change Required']}</td>"
        html += f"<td style='padding:5px'><b>{status_icon}</b></td></tr>"
        
    html += "</table>"
    display(HTML(f"<h3>📊 Status Summary</h3><p>Saved: <code>{s_file}</code></p>" + html))

In [ ]:
# === Cell 6: Email Control (Overdue Logic) ===
import ipywidgets as widgets
from urllib.parse import quote
from datetime import datetime, timedelta
import requests

def get_email(name): 
    try:
        clean = quote(name.split("(")[0].strip())
        url = f"https://graph.microsoft.com/v1.0/users?$filter=startswith(displayName,'{clean}')&$select=mail,userPrincipalName"
        res = requests.get(url, headers=headers).json().get('value')
        if res: return res[0].get('mail') or res[0].get('userPrincipalName')
    except: pass
    return ""

def calculate_due_date(iso_date_str):
    """Calculates Version 1 Date + 14 Days"""
    if not iso_date_str or pd.isna(iso_date_str): return "ASAP"
    try:
        # Parse ISO format (Created Date)
        dt = datetime.strptime(str(iso_date_str)[:19], "%Y-%m-%dT%H:%M:%S")
        due = dt + timedelta(days=14)
        
        # Format: Dec 24th, 2025
        day = due.day
        if 11 <= day <= 13: suffix = "th"
        else: suffix = {1: "st", 2: "nd", 3: "rd"}.get(day % 10, "th")
        return due.strftime(f"%b {day}{suffix}, %Y")
    except Exception as e:
        return "ASAP"

if 'summary_df' not in globals() or len(summary_df) == 0:
    print("No data to process.")
else:
    targets = summary_df[summary_df['missing'] > 0].copy()
    
    style = {'description_width': 'initial'}
    # Overdue Subject
    subj = widgets.Text(value="[Overdue] Action Required: Access Entitlement Review", description="Subject:", layout=widgets.Layout(width='95%'))
    
    # Overdue Template
    default_tpl = (
        "Hi {name},\n\n"
        "This is a reminder that the access review for {app_name} is now overdue.\n"
        "The review was due on {due_date}.\n\n"
        "Pending records: {missing}\n"
        "Link: {link}\n\n"
        "Please complete this immediately."
    )
    
    tpl = widgets.Textarea(value=default_tpl, description="Template:", layout=widgets.Layout(width='80%', height='100px'))
    btn_apply_template = widgets.Button(description="🔄 Apply Template to All", button_style='info', layout=widgets.Layout(width='180px', height='100px'))
    
    rows = []
    row_map = {}
    print(f"🔍 Searching emails for {len(targets)} reviewers...")
    
    for _, r in targets.iterrows():
        key = f"{r['App_Name']}_{r['reviewer']}"
        email = get_email(r['reviewer'])
        due_date_str = calculate_due_date(r.get('file_date')) # Using Created_Date from base
        
        row_context = {'name': r['reviewer'], 'missing': r['missing'], 'link': r['folder_url'], 'app_name': r['App_Name'], 'due_date': due_date_str}
        
        w_chk = widgets.Checkbox(value=True, layout=widgets.Layout(width='30px'))
        w_info = widgets.HTML(
            f"<div style='line-height:1.2; font-size:13px'><b>{r['App_Name']}</b><br>{r['reviewer']}<br>"
            f"<span style='color:red'>Miss: {r['missing']}</span><br>"
            f"<span style='color:darkred; font-weight:bold; font-size:11px'>Due: {due_date_str}</span></div>", 
            layout=widgets.Layout(width='180px')
        )
        w_to = widgets.Text(value=email, placeholder="To", layout=widgets.Layout(width='200px'))
        w_body = widgets.Textarea(value=default_tpl.format(**row_context), layout=widgets.Layout(width='95%', height='60px'))
        w_btn = widgets.Button(description="Send", layout=widgets.Layout(width='60px'), button_style='warning')
        
        box = widgets.HBox([w_chk, w_info, w_to, w_body, w_btn], layout=widgets.Layout(border='1px solid #eee', padding='10px', margin='5px 0', align_items='center', min_height='90px'))
        rows.append(box)
        row_map[key] = {'chk': w_chk, 'to': w_to, 'body': w_body, 'btn': w_btn, 'ctx': row_context}
        
        def mk_send(k, from_addr=SENDER_EMAIL):
            def snd(b):
                d = row_map[k]
                d['btn'].disabled = True
                d['btn'].description = "..."
                url = f"https://graph.microsoft.com/v1.0/users/{from_addr}/sendMail"
                payload = {"message": {"subject": subj.value, "body": {"contentType": "HTML", "content": d['body'].value.replace("\n", "<br>")}, "toRecipients": [{"emailAddress": {"address": d['to'].value}}]}}
                try:
                    res = requests.post(url, headers={**headers, "Content-Type": "application/json"}, json=payload)
                    if res.status_code == 202: 
                        d['btn'].button_style='success'; d['btn'].description='OK'; d['chk'].value=False
                    else: 
                        d['btn'].button_style='danger'; d['btn'].description='Err'; d['btn'].disabled=False
                        print(f"Error ({k}): {res.text}")
                except Exception as e:
                    d['btn'].button_style='danger'; d['btn'].description='Err'; d['btn'].disabled=False
                    print(f"Ex ({k}): {e}")
            return snd
        w_btn.on_click(mk_send(key))

    def on_apply_template(b):
        current_tpl = tpl.value
        count = 0
        for k, d in row_map.items():
            if d['chk'].value:
                try:
                    d['body'].value = current_tpl.format(**d['ctx'])
                    count += 1
                except: pass
        print(f"✅ Template applied to {count} rows.")

    btn_apply_template.on_click(on_apply_template)

    clear_output()
    header = widgets.VBox([widgets.HTML("<h3>📧 Overdue Notice Sender</h3>"), subj, widgets.HBox([tpl, btn_apply_template], layout=widgets.Layout(align_items='center')), widgets.HTML("<i>Placeholders: {name}, {missing}, {link}, {app_name}, {due_date}</i>")])
    display(widgets.VBox([header, widgets.VBox(rows, layout=widgets.Layout(max_height='600px', overflow_y='auto'))]))

In [ ]:
import requests
import json

def retry_copilot_api():
    print("⚔️ 回合 2：權限補齊後，再次挑戰 Copilot Endpoint...")
    
    # 這是 Graph API 呼叫 Copilot 的標準進入點
    url = "https://graph.microsoft.com/beta/copilot/conversations"
    
    # 這裡嘗試帶入一個簡單的 prompt，有些 API 實作需要 body
    payload = {
        "promptText": "Hello Copilot, who am I?" 
    }
    
    try:
        response = requests.post(url, headers=headers, json=payload)
        
        print(f"📡 Status Code: {response.status_code}")
        
        if response.status_code in [200, 201]:
            print("🎉🎉🎉 成功了！我們連上了！")
            data = response.json()
            print(json.dumps(data, indent=2, ensure_ascii=False))
            return data
        else:
            # 如果還是失敗，看新的錯誤訊息
            print(f"❌ 依然失敗: {response.text}")
            
    except Exception as e:
        print(f"💥 發生錯誤: {e}")

# 執行
retry_copilot_api()

In [6]:
# === Cell 4 (V12b): M365 Copilot 同步版 + 500 重試 & 詳細錯誤 ===
import ipywidgets as widgets
from IPython.display import display
import requests, json, time, re, uuid

GRAPH_ROOT = "https://graph.microsoft.com/beta"
# 你先前定義的 headers 至少要含 Authorization；這裡補 Content-Type
headers["Content-Type"] = "application/json"

debug_log = []
def log(msg):
    timestamp = time.strftime("%H:%M:%S")
    debug_log.append(f"[{timestamp}] {msg}")

def clean_bot_text(text):
    if not text: return ""
    text = text.replace("\n", "<br>")
    text = re.sub(r'\[(.*?)\]\((.*?)\)',
                  r'<a href="\\2" target="_blank" style="color:#0078d4">\\1</a>', text)
    return text

conv_id = None
def ensure_conversation():
    global conv_id
    if conv_id: return conv_id
    log("➕ 建立新對話")
    r = requests.post(f"{GRAPH_ROOT}/copilot/conversations", headers=headers, json={}, timeout=15)
    r.raise_for_status()
    conv_id = r.json()["id"]
    log(f"✅ conv_id: {conv_id}")
    return conv_id

def post_chat(cid, body):
    # 針對 500 做最多 3 次重試（指數退避）
    for attempt in range(3):
        try:
            r = requests.post(f"{GRAPH_ROOT}/copilot/conversations/{cid}/chat",
                              headers=headers, json=body, timeout=30)
            log(f"📥 Status: {r.status_code}")
            if r.status_code < 500:
                return r  # 成功或 4xx 直接回傳
        except Exception as e:
            log(f"⚠️ Exception: {e}")
        sleep = 2 ** attempt
        log(f"🔁 500/Exception，{sleep}s 後重試 ({attempt+1}/3)")
        time.sleep(sleep)
    return r  # 回傳最後一次結果

def chat_once(user_text, tz="America/Los_Angeles"):
    cid = ensure_conversation()
    body = {
        "message": { "text": user_text },
        "locationHint": { "timeZone": tz }  # 必填
        # 若要關掉 Web 搜尋可加：
        # "contextualResources": { "webContext": { "isWebEnabled": False } }
    }
    log("✉️ POST /chat")
    r = post_chat(cid, body)

    try:
        r.raise_for_status()
    except Exception:
        # 把更多細節塞回 UI
        err_json = ""
        try:
            err_json = json.dumps(r.json(), ensure_ascii=False)
        except:
            err_json = r.text
        req_id = r.headers.get("request-id", "n/a")
        cli_id = r.headers.get("client-request-id", "n/a")
        return f"❌ HTTP {r.status_code}<br>request-id: {req_id}<br>client-request-id: {cli_id}<br>body: {err_json}"

    msgs = r.json().get("messages", [])
    if not msgs: return "(empty response)"
    return msgs[-1].get("text", "(no text)")

# ========= UI =========
chat_history_list = []
ui_style = """<style> .chat-app { font-family:'Segoe UI',sans-serif;max-width:800px;margin:0 auto; }
.chat-window { background:#fff;border:1px solid #e0e0e0;border-radius:8px;height:400px;overflow-y:auto;padding:20px; }
.msg-row { margin-bottom:15px;display:flex;flex-direction:column; }
.msg-user { align-items:flex-end; } .msg-bot { align-items:flex-start; }
.bubble { padding:10px 16px;max-width:80%;line-height:1.5;font-size:15px;position:relative;word-wrap:break-word; }
.bubble-user { background:#0078D4;color:white;border-radius:18px 18px 4px 18px; }
.bubble-bot { background:#f3f2f1;color:#201f1e;border-radius:18px 18px 18px 4px;border:1px solid #e1dfdd; }
.debug-box { font-family:monospace;font-size:11px;color:#666;background:#f0f0f0;border:1px dashed #ccc;
             padding:8px;margin-top:10px;height:120px;overflow-y:auto; } code { background:#e1dfdd;padding:2px 6px;
             border-radius:3px;font-size:12px; } </style>"""

header_w = widgets.HTML("<h2>🤖 M365 Copilot (V12b - 同步 API)</h2>")
history_w = widgets.HTML(value=ui_style + "<div class='chat-app'><div class='chat-window'></div></div>")
debug_w = widgets.HTML(value="<div class='debug-box'>Ready.</div>")
input_w = widgets.Text(placeholder="試試輸入: Hello", layout=widgets.Layout(width='80%'))
send_btn_w = widgets.Button(description="Send", button_style='primary', layout=widgets.Layout(width='18%'))
loading_w = widgets.HTML(value="", layout=widgets.Layout(height='30px'))

def render_chat():
    history_w.value = f"{ui_style}<div class='chat-app'><div class='chat-window'>{''.join(chat_history_list)}</div></div>"

def update_debug():
    debug_w.value = "<div class='debug-box'>" + "<br>".join(debug_log[-12:]) + "</div>"

def on_chat_submit(b):
    user_text = input_w.value.strip()
    if not user_text: return
    debug_log.clear(); log("🏁 User send")
    chat_history_list.append(f"<div class='msg-row msg-user'><div class='bubble bubble-user'>{user_text}</div></div>")
    render_chat(); update_debug()
    input_w.value = ""; input_w.disabled = True; send_btn_w.disabled = True
    loading_w.value = "<span style='color:#0078D4; margin-left:10px;'>⏳ Copilot 正在回覆...</span>"
    try:
        reply = chat_once(user_text)
    except Exception as e:
        reply = f"❌ Error: {e}"
        log(str(e))
    update_debug()
    chat_history_list.append(f"<div class='msg-row msg-bot'><div class='bubble bubble-bot'>{clean_bot_text(reply)}</div></div>")
    render_chat()
    loading_w.value = ""; input_w.disabled = False; send_btn_w.disabled = False

send_btn_w.on_click(on_chat_submit)
input_w.on_submit(on_chat_submit)

display(widgets.VBox([header_w, history_w, loading_w, input_w, send_btn_w, debug_w]))

/var/folders/zv/g0jcxzqd3qngjqkxxg14pdv40000gp/T/ipykernel_49330/2478190679.py:122: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  input_w.on_submit(on_chat_submit)
